In [ ]:
# -----------------------------------------------
# 품질시험 성적서 데이터 전처리 전체 코드 (20250807) BY 정경문
# -----------------------------------------------

import pandas as pd
import numpy as np
import re
from datetime import datetime

# [1] 데이터 불러오기 및 컬럼 정리
# -------------------------------------------------
df = pd.read_excel('C:/Users/146058/CSI_DATA/QT_Cert_4.xlsx')
df = df.drop(columns=["공사명"])
df = df.rename(columns={"공사명.1": "공사명"})


# [2] 텍스트 정제 함수 정의
# -------------------------------------------------
def clean_text(text):
    text = re.sub(r"[\n\t]", " ", text)
    text = re.sub(r"\(.*?\)", "", text)
    text = re.sub(r"\[.*?\]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def preprocess_dataframe(df):
    df_cleaned = df.copy()
    df_cleaned['성적서번호'] = df_cleaned['시험실'].str.extract(r"\[(.*?)\]")
    df_cleaned['시험실'] = df_cleaned['시험실'].astype(str).str.replace(r"\[.*?\]", "", regex=True)
    for col in df_cleaned.columns:
        if col not in ['시험실', '시공자', '공사명']:
            df_cleaned[col] = df_cleaned[col].apply(lambda x: clean_text(x) if isinstance(x, str) else x)
    return df_cleaned

cleaned_df = preprocess_dataframe(df)


# [3] 결측값 처리 및 특정 단어 제거
# -------------------------------------------------
cleaned_df.replace("nan", np.nan, inplace=True)
cleaned_df.fillna(method='ffill', inplace=True)

words_to_remove = ["S1", "S2", "S3", "S4", "S5", "S6", "대한민국"]
columns_to_clean = ["시료명(생산국)", "시료 채취 장소", "규격", "종목", "결과"]
for word in words_to_remove:
    cleaned_df[columns_to_clean] = cleaned_df[columns_to_clean].apply(
        lambda col: col.str.replace(word, "", regex=False)
    )


# [4] 시료명 중복 제거 및 표준화
# -------------------------------------------------
def dedup_sample_names(sample_str):
    if pd.isnull(sample_str): return sample_str
    items = [item.strip() for item in sample_str.split(',')]
    seen = set()
    return ', '.join([item for item in items if item and not (item in seen or seen.add(item))])

cleaned_df["시료명(생산국)"] = cleaned_df["시료명(생산국)"].apply(dedup_sample_names)

sample_name_map = {
    "철근 콘크리트용 봉강": ["철근콘크리트용 봉강", "철근콘크리트용봉강", "철근 콘크리트용 봉강", "철근콘크리트 봉강", "이형철근", "철근", "철근#1"],
    "성토용 흙": ["성토용 흙", "성토용흙", "토사"],
    "콘크리트용 부순 굵은 골재": ["콘크리트용 부순 굵은 골재", "콘크리트용 부순굵은골재", "부순굵은골재", "콘크리트용 굵은골재", "굵은골재", "콘크리트용 굵은골재쌍용"],
    "콘크리트용 부순 잔골재": ["콘크리트용 잔골재", "콘크리트용 골재", "콘크리트용 부순잔골재", "잔골재", "부순잔골재", "콘크리트용 부순골재", "콘크리트용 잔골재쌍용"],
    "굳지 아니한 콘크리트": ["레미콘", "콘크리트", "레디믹스트 콘크리트", "굳지아니한 콘크리트", "레디믹스트콘크리트", "굳지않은 콘크리트"],
    "시멘트": ["시멘트", "포틀랜드 시멘트 1종", "포틀랜드시멘트", "시멘트#1"],
    "평판재하시험": ["평판재하"],
    "동재하시험": ["동재하", "PHC파일 동재하시험"],
    "구조물 기초" : ["구조물기초", "지하 7층 구조물 기초부위", "구조물 기초 평판재하", "구조물 기초 평판 지지력 시험", "구조물 기초 지반", "구조물기초 평판재하", "구조물 기초, 구조물기초","구조물기초 평판재하시험","'구조물기초 #1, 구조물기초#2'" , "구조물기초초"],
    "도자기질 타일": ["도기질타일"],
    "보조기층": ["보조기층재"],
    "콘크리트 벽돌": ["콘크리트벽돌"],
    "강관말뚝":["강관파일"],
    "PHC파일":["PHC 파일"],
    "어스앵커": ["어스앵커 인장시험","어스 앵커"],
    "구조물용 토크전단형 고장력볼트": ["롯데몰 송도2단계 중 PRD 철골공사 토오크 전단형 고장력볼트"],
    "철근 커플러": ["커플러"],
    "일반구조용 압연강재": ["일반구조용압연강재"],
    "굳은 콘크리트": ["콘크리트 공시체", "굳은콘크리트", "공시체"],
    "아스팔트 콘크리트": ["아스팔트콘크리트","아스콘"],
    "구조물용 토크전단형 고장력볼트" : ["롯데몰 송도2단계 중 PRD 철골공사 토오크 전단형 고장력볼트"]
    
}

replace_dict = {
    v.replace(" ", ""): rep
    for rep, variants in sample_name_map.items()
    for v in variants
}

def normalize_sample_name(sample_str):
    if pd.isnull(sample_str): return sample_str
    items = [item.strip() for item in sample_str.split(',')]
    seen, normalized = set(), []
    for item in items:
        key = item.replace(" ", "")
        new_item = replace_dict.get(key, item)
        if new_item not in seen:
            seen.add(new_item)
            normalized.append(new_item)
    return ', '.join(normalized)

cleaned_df["시료명(생산국)"] = cleaned_df["시료명(생산국)"].apply(normalize_sample_name)


# [5] 결과값 분리 처리
# -------------------------------------------------
expanded_rows = []
for _, row in cleaned_df.iterrows():
    result_text = str(row['결과'])
    if '·' in result_text and ':' in result_text:
        entries = [item.strip() for item in result_text.split('·') if item.strip()]
        for entry in entries:
            try:
                key, value = entry.split(':')
                new_row = row.copy()
                new_row['시험종목'] = key.strip().lower()
                new_row['결과값'] = value.strip()
                expanded_rows.append(new_row)
            except ValueError:
                continue
    else:
        row['시험종목'] = row['종목']
        row['결과값'] = row['결과']
        expanded_rows.append(row)

df_expanded = pd.DataFrame(expanded_rows)


# [6] 날짜형 처리
# -------------------------------------------------
df_expanded["발급일자"] = pd.to_datetime(df_expanded["발급일자"], errors="coerce")
df_expanded["발급일자"] = df_expanded["발급일자"].dt.strftime("%Y-%m-%d")


# [7] 결과 확인 또는 저장
# -------------------------------------------------
# print(df_expanded.head())
# df_expanded.to_csv('output_path.csv', index=False, encoding='utf-8-sig')
# df_expanded

In [4]:
df_expanded

,NO,발급번호,시험실,발급일자,시료명(생산국),시료 채취 장소,규격,성과 이용 목적,공사명,발주자,...,국가시설여부,종목,방법,결과,비고,생산자정보,품질시험성적서 링크,성적서번호,시험종목,결과값
0,1.0,IS-2025-025942-00,한국건설표준연구원(주),2025-02-22,아스콘공시체,현장내,표층,품질시험·검사 대행,2024년 상습침수지역 배수로 정비사업(대정읍 무릉리),제주특별자치도서귀포시,...,해당없음,NaN,NaN,NaN,NaN,NaN,NaN,제 제주-3-19호,NaN,NaN
1,1.0,IS-2025-025942-00,한국건설표준연구원(주),2025-02-22,아스콘공시체,현장내,표층,품질시험·검사 대행,2024년 상습침수지역 배수로 정비사업(대정읍 무릉리),제주특별자치도서귀포시,...,해당없음,체가름,KS F 2502,· 20mm 체 통과율 : 100 · 13mm 체 통과율 : 100 · 10mm 체...,NaN,NaN,NaN,제 제주-3-19호,20mm 체 통과율,100
1,1.0,IS-2025-025942-00,한국건설표준연구원(주),2025-02-22,아스콘공시체,현장내,표층,품질시험·검사 대행,2024년 상습침수지역 배수로 정비사업(대정읍 무릉리),제주특별자치도서귀포시,...,해당없음,체가름,KS F 2502,· 20mm 체 통과율 : 100 · 13mm 체 통과율 : 100 · 10mm 체...,NaN,NaN,NaN,제 제주-3-19호,13mm 체 통과율,100
1,1.0,IS-2025-025942-00,한국건설표준연구원(주),2025-02-22,아스콘공시체,현장내,표층,품질시험·검사 대행,2024년 상습침수지역 배수로 정비사업(대정읍 무릉리),제주특별자치도서귀포시,...,해당없음,체가름,KS F 2502,· 20mm 체 통과율 : 100 · 13mm 체 통과율 : 100 · 10mm 체...,NaN,NaN,NaN,제 제주-3-19호,10mm 체 통과율,91
1,1.0,IS-2025-025942-00,한국건설표준연구원(주),2025-02-22,아스콘공시체,현장내,표층,품질시험·검사 대행,2024년 상습침수지역 배수로 정비사업(대정읍 무릉리),제주특별자치도서귀포시,...,해당없음,체가름,KS F 2502,· 20mm 체 통과율 : 100 · 13mm 체 통과율 : 100 · 10mm 체...,NaN,NaN,NaN,제 제주-3-19호,5mm 체 통과율,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60488,19.0,IS-2025-111824-00,(주)건설품질안전평가원,2025-07-06,"철근 가스 압접 이음 SD600 D22, D29mm",현장내,"SD600 D22, D29mm",품질시험·검사 대행,마곡D18-B BL R&D센터(지식산업센터) 신축공사,신한자산신탁,...,해당없음,고사이클반복시험,KS B 0554,· 잔류변형량 : 0.13 · 파단유무 : 파단되지 않음 · 잔류변형량 : 0.18...,강종 : SD400,생산자,https://www.csi.go.kr/cmq/com/reportViewPop.do...,경기 1-3-108호,파단유무,파단되지 않음
60489,20.0,IS-2025-111823-00,교정시험실,2025-07-06,솔플레이트 용접검사,"복술1교, 대둔ic교, 현리교",-,품질시험·검사 대행,국도42호선 복술1교 등 3개소 시설물 정비공사,홍천국토관리사무소,...,해당없음,자기비파괴검사,KS D 0213,· 복술1교 SOLE PLATE : 합격 · 대둔IC교 SOLE PLATE : 합격...,강종 : SD400,-,https://www.csi.go.kr/cmq/com/reportViewPop.do...,광주-3-5호,복술1교 sole plate,합격
60489,20.0,IS-2025-111823-00,교정시험실,2025-07-06,솔플레이트 용접검사,"복술1교, 대둔ic교, 현리교",-,품질시험·검사 대행,국도42호선 복술1교 등 3개소 시설물 정비공사,홍천국토관리사무소,...,해당없음,자기비파괴검사,KS D 0213,· 복술1교 SOLE PLATE : 합격 · 대둔IC교 SOLE PLATE : 합격...,강종 : SD400,-,https://www.csi.go.kr/cmq/com/reportViewPop.do...,광주-3-5호,대둔ic교 sole plate,합격
60489,20.0,IS-2025-111823-00,교정시험실,2025-07-06,솔플레이트 용접검사,"복술1교, 대둔ic교, 현리교",-,품질시험·검사 대행,국도42호선 복술1교 등 3개소 시설물 정비공사,홍천국토관리사무소,...,해당없음,자기비파괴검사,KS D 0213,· 복술1교 SOLE PLATE : 합격 · 대둔IC교 SOLE PLATE : 합격...,강종 : SD400,-,https://www.csi.go.kr/cmq/com/reportViewPop.do...,광주-3-5호,현리교 soleplate,합격


In [ ]:
# 이하, fuzz를 활용한 유사 항목 자동 분류: cleaned_df['종목'] -> 표준명 매핑 코드입니다. '종목', '시료명(생산국)' 컬럼이 표준화 대상입니다. 

import unicodedata
from rapidfuzz import fuzz
import pprint

# 1) 텍스트 정규화 함수 (공백/특수문자/대소문/한글 정규화)
def normalize_text(s: str) -> str:
    if pd.isna(s):
        return ""
    s = str(s)
    s = unicodedata.normalize("NFKC", s)  # 유니코드 정규화
    s = s.strip()
    # 흔한 변형 감소: 공백/특수문자 제거, 전각/반각/대소문 통일
    rm = "".join(ch for ch in s if ch.isalnum())  # 한글/영문/숫자만 남김
    return rm.lower()

# 2) 고유값 수집
raw_values = cleaned_df['종목'].dropna().astype(str).str.strip()
uniques = raw_values.unique().tolist()

# 3) 원본 -> 정규화 매핑
norm_map = {u: normalize_text(u) for u in uniques}

# 4) 퍼지 매칭으로 클러스터링 (그리디, O(n^2) 근사)
#    - threshold: 85(권장). 너무 빡세면 80~83으로 낮추세요.
threshold = 85
unused = set(uniques)
clusters = []  # 각 클러스터는 원본 문자열 리스트

while unused:
    seed = unused.pop()
    seed_norm = norm_map[seed]
    group = [seed]

    # 후보군 스냅샷 후 순회
    rest = list(unused)
    for cand in rest:
        score = fuzz.token_set_ratio(seed_norm, norm_map[cand])
        if score >= threshold:
            group.append(cand)
            unused.remove(cand)
    clusters.append(group)

# 5) 각 클러스터의 "표준명" 선택 규칙
#    - cleaned_df 전체에서 등장 빈도가 가장 높은 원본을 대표로
#    - 동률일 땐 길이가 긴 것(정보량 많음)을 대표로
vc = raw_values.value_counts()

def pick_representative(group):
    # 빈도 높은 순, 길이 긴 순
    group_sorted = sorted(
        group,
        key=lambda x: (vc.get(x, 0), len(x)),
        reverse=True
    )
    return group_sorted[0]

# 6) 표준명 사전 생성 (sample_name_map 형식)
sample_name_map_auto = {
    pick_representative(group): sorted(group, key=lambda x: (vc.get(x, 0), len(x)), reverse=True)
    for group in clusters
}

# 7) 원본 -> 표준명 매핑 딕셔너리 생성
value_to_standard = {}
for standard, members in sample_name_map_auto.items():
    for m in members:
        value_to_standard[m] = standard

# 8) 데이터프레임에 적용
cleaned_df['종목_표준명'] = cleaned_df['종목'].map(lambda x: value_to_standard.get(str(x).strip(), x))

# # 9) 확인용 출력 (상위 몇 개만)
# print("클러스터 개수:", len(sample_name_map_auto))
# for k, v in list(sample_name_map_auto.items())[:10]:
#     print(f"[표준명] {k} -> {v[:8]}{'...' if len(v)>8 else ''}")
# 11) 표준명: [변형들] 형태로 전체 매핑 출력

pp = pprint.PrettyPrinter(width=200, compact=False)
pp.pprint(sample_name_map_auto)

# print("\n=== 표준명 매핑 결과 (Key-Value) ===")
# for standard, variants in sample_name_map_auto.items():
#     print(f'"{standard}": {variants}')


# 필요하면 엑셀/CSV로 저장
# cleaned_df.to_csv("종목_표준화_결과.csv", index=False, encoding="utf-8-sig")


In [ ]:
# fuzz로 정리된 리스트입니다. 

{'0.08mm 체 통과량': ['0.08mm 체 통과량'],
 '0.08mm 체 통과율': ['0.08mm 체 통과율', '0.08mm 통과율'],
 '0.08밀리미터 통과량': ['0.08밀리미터 통과량'],
 '0.08밀리미터체 통과량': ['0.08밀리미터체 통과량', '골재의 0.08밀리미터체 통과량'],
 '0.2% 영구연신율에 대한 하중': ['0.2% 영구연신율에 대한 하중'],
 '0.2% 항복 강도': ['0.2% 항복 강도'],
 '0.2％ 영구 변형시 응력': ['0.2％ 영구 변형시 응력'],
 '1,2-디클로로에탄': ['1,2-디클로로에탄', '1.2-디트로로에탄'],
 '1.2밀리미터체 잔류율': ['1.2밀리미터체 잔류율'],
 '1축 압축강도': ['1축 압축강도'],
 '28일': ['28일'],
 '2종': ['2종'],
 '3M 직선자': ['3M 직선자'],
 '3종/내모르타르성': ['3종/내모르타르성'],
 '3종/내습성': ['3종/내습성'],
 '3종/내충격성': ['3종/내충격성'],
 '3종/도막경도': ['3종/도막경도'],
 '3종/부착성': ['3종/부착성'],
 '3종/아세트산-염수분무시험': ['3종/아세트산-염수분무시험'],
 '3종/입자분포': ['3종/입자분포'],
 '3축 압축시험': ['3축 압축시험'],
 '45도, 0도 확산반사율': ['45도, 0도 확산반사율'],
 '5% 인장변형률에서의 인장강도': ['5% 인장변형률에서의 인장강도', '10% 인장변형률에서의 인장강도'],
 '5mm체 통과율': ['5mm체 통과율'],
 '6.9N/㎟ 하중시 신장률': ['6.9N/㎟ 하중시 신장률'],
 '60° 경면 광택도': ['60° 경면 광택도'],
 '6가크롬': ['6가크롬'],
 '7일': ['7일'],
 '90도 박리접착강도': ['90도 박리접착강도'],
 'BOD': ['BOD'],
 'C,Si,Mn,P,S,Ni,Cr': ['C,Si,Mn,P,S,Ni,Cr'],
 'CBR시험': ['CBR시험'],
 'D15': ['D15'],
 'D85': ['D85'],
 'E': ['E'],
 'EPS': ['EPS'],
 'EVA': ['EVA'],
 'G*/sinδ': ['G*/sinδ', 'G*·sinδ'],
 'IRI': ['IRI'],
 'KS F 2591': ['KS F 2591'],
 'KS F 5650': ['KS F 5650'],
 'PG': ['PG'],
 'PG등급': ['PG등급'],
 'Ph변화': ['Ph변화'],
 'Pipe Stiffness': ['Pipe Stiffness'],
 'RTFO 후 동적전단': ['RTFO 후 동적전단'],
 'RTFO후의 점도비': ['RTFO후의 점도비'],
 'RTFO후의 중량변화율': ['RTFO후의 중량변화율'],
 'SB-1': ['SB-1'],
 'Stiffness': ['Stiffness'],
 'TOC': ['TOC'],
 'U.V코팅층 확인': ['U.V코팅층 확인'],
 'UV 노화 전': ['UV 노화 전'],
 'UV 노화 후': ['UV 노화 후'],
 'UV 노화시험': ['UV 노화시험'],
 'UV변색유무': ['UV변색유무'],
 'VOCs': ['VOCs'],
 'V형새김된 철근의 소성연신율': ['V형새김된 철근의 소성연신율'],
 'm-value': ['m-value'],
 'm56/m28': ['m56/m28'],
 'pH': ['pH'],
 'vE': ['vE'],
 '· - : -': ['· - : -'],
 '· 101동 #446 : 1844.7': ['· 101동 #446 : 1844.7'],
 '· 길이 : 0.1 · 너비 : 0.1': ['· 길이 : 0.1 · 너비 : 0.1'],
 '· 두께 : 12.25 · 너비 : 606.0 · 길이 : 1 819.7 · 대각선의 길이차 : 0.0': ['· 두께 : 12.25 · 너비 : 606.0 · 길이 : 1 819.7 · 대각선의 길이차 : 0.0'],
 '· 충격 시험 : 깨짐 없음': ['· 충격 시험 : 깨짐 없음'],
 '· 하중비 : 0.8': ['· 하중비 : 0.8'],
 '· 휨 강성 변형량 : 22.99': ['· 휨 강성 변형량 : 22.99'],
 '가로': ['가로'],
 '가사 시간, 부착가능 시간': ['가사 시간, 부착가능 시간'],
 '가사시간': ['가사시간'],
 '가수분해저항성': ['가수분해저항성'],
 '가스유해성': ['가스유해성'],
 '가시광선 투과율': ['가시광선 투과율'],
 '가열 신축 성상': ['가열 신축 성상'],
 '가열감량률': ['가열감량률'],
 '가열길이변화율': ['가열길이변화율'],
 '가열노화시험': ['가열노화시험'],
 '가열변화': ['가열변화'],
 '가열신축성상-신축률': ['가열신축성상-신축률'],
 '가열에 의한 길이변화율': ['가열에 의한 길이변화율'],
 '가열잔분': ['가열잔분'],
 '가열후 동탄성계수': ['가열후 동탄성계수'],
 '가열후 손실계수': ['가열후 손실계수'],
 '가열후 치수안정성': ['가열후 치수안정성'],
 '간극률': ['간극률'],
 '간극율': ['간극율'],
 '간접인장강도': ['간접인장강도'],
 '갈라짐': ['갈라짐'],
 '감수율': ['감수율'],
 '강성 하중 : 98 N 지점 간 거리 : 1 200 mm': ['강성 하중 : 98 N 지점 간 거리 : 1 200 mm'],
 '강열 감량': ['강열 감량'],
 '개공률': ['개공률'],
 '개방시간': ['개방시간'],
 '개폐 반복성': ['개폐 반복성'],
 '개폐력': ['개폐력'],
 '거푸집의 처짐 시험': ['거푸집의 처짐 시험'],
 '건전도시험': ['건전도시험'],
 '건전성': ['건전성'],
 '건조도막 두께': ['건조도막 두께'],
 '건조도막의 상태': ['건조도막의 상태'],
 '건조수축': ['건조수축'],
 '건조시간': ['건조시간'],
 '건조시간, 재도장시험': ['건조시간, 재도장시험'],
 '걸침 고리의 본체, 부착부의 전단 하중': ['걸침 고리의 본체, 부착부의 전단 하중'],
 '걸침 고리의 이탈 방지 전단 하중': ['걸침 고리의 이탈 방지 전단 하중'],
 '검증시험': ['검증시험'],
 '겉모양': ['겉모양'],
 '겉모양 및 치수': ['겉모양 및 치수'],
 '겉모양, 두께': ['겉모양, 두께'],
 '겉모양, 모양': ['겉모양, 모양'],
 '겉모양, 모양, 치수': ['겉모양, 모양, 치수', '겉모양, 모양, 치수, 무게'],
 '겉모양, 치수': ['겉모양, 치수', '모양, 치수'],
 '겉모양, 치수, 무게': ['겉모양, 치수, 무게', '모양, 치수, 무게'],
 '겉모양, 치수, 밀도': ['겉모양, 치수, 밀도'],
 '겉모양, 치수, 직각도': ['겉모양, 치수, 직각도'],
 '겉보기 밀도': ['겉보기 밀도'],
 '겉보기비중': ['겉보기비중'],
 '겔 타임': ['겔 타임'],
 '격자크기': ['격자크기'],
 '결로방지성능': ['결로방지성능'],
 '결하부 전단하중': ['결하부 전단하중'],
 '경도': ['경도'],
 '경도시험': ['경도시험'],
 '경량편': ['경량편'],
 '경량편 시험': ['경량편 시험'],
 '경시변화량': ['경시변화량', '경시변화'],
 '경시변화에 따른 공기량변화': ['경시변화에 따른 공기량변화'],
 '경화건조': ['경화건조'],
 '경화건조시간': ['경화건조시간'],
 '경화수축률': ['경화수축률'],
 '계단 발판 처짐량': ['계단 발판 처짐량'],
 '고응력 인장 압축 반복 시험': ['고응력 인장 압축 반복 시험'],
 '고응력반복내력시험': ['고응력반복내력시험'],
 '고체비중': ['고체비중'],
 '고형분': ['고형분', '고형분량', '총 고형분'],
 '고형분량의 차': ['고형분량의 차'],
 '고화건조시간': ['고화건조시간'],
 '골재 알칼리 잠재반응': ['골재 알칼리 잠재반응'],
 '골재의 밀도, 흡수율': ['골재의 밀도, 흡수율', '골재 밀도 및 흡수율'],
 '골재의 체가름': ['골재의 체가름'],
 '곰팡이저항성': ['곰팡이저항성'],
 '공극률': ['공극률'],
 '공기기포상태': ['공기기포상태'],
 '공기량': ['공기량'],
 '공기량의 변화': ['공기량의 변화'],
 '공시체제작': ['공시체제작'],
 '공칭단면적': ['공칭단면적'],
 '공칭지름': ['공칭지름'],
 '관벽 두께': ['관벽 두께'],
 '관의 강성': ['관의 강성'],
 '관의 색': ['관의 색'],
 '관입량시험': ['관입량시험'],
 '광택': ['광택'],
 '광택도': ['광택도'],
 '광학 박막 성능의 방사율': ['광학 박막 성능의 방사율'],
 '구리': ['구리'],
 '구멍의 크기': ['구멍의 크기'],
 '구멍지름': ['구멍지름'],
 '구상화율': ['구상화율'],
 '구재 아스팔트 침입도': ['구재 아스팔트 침입도'],
 '굴곡 강도': ['굴곡 강도'],
 '굴곡 성능': ['굴곡 성능', '내굴곡성', '굴곡성'],
 '굴곡 저항 성능': ['굴곡 저항 성능'],
 '굴곡 탄성률': ['굴곡 탄성률'],
 '굴곡각도': ['굴곡각도'],
 '굴곡시험': ['굴곡시험'],
 '굴곡파괴하중': ['굴곡파괴하중'],
 '굴절률': ['굴절률'],
 '굵은 골재의 마모율': ['굵은 골재의 마모율'],
 '굵은골재 파쇄면 비율': ['굵은골재 파쇄면 비율'],
 '굽음': ['굽음'],
 '굽음 또는 비틀림': ['굽음 또는 비틀림'],
 '굽힘강도': ['굽힘강도'],
 '굽힘검사': ['굽힘검사'],
 '굽힘성': ['굽힘성'],
 '굽힘시험': ['굽힘시험'],
 '굽힘정도': ['굽힘정도'],
 '굽힘탄성률': ['굽힘탄성률'],
 '균열': ['균열'],
 '균열 저항성': ['균열 저항성'],
 '균열검사': ['균열검사'],
 '균열발생 저항성': ['균열발생 저항성'],
 '극한인장강도': ['극한인장강도'],
 '극한전단강도': ['극한전단강도'],
 '기건 비중': ['기건 비중'],
 '기건밀도': ['기건밀도'],
 '기계적성질': ['기계적성질'],
 '기계적이음＞인장강도': ['기계적이음＞인장강도'],
 '기계적이음＞항복강도': ['기계적이음＞항복강도'],
 '기름기포상태': ['기름기포상태'],
 '기름성분': ['기름성분'],
 '기밀성': ['기밀성'],
 '기층용, 중간층용 - 안정도': ['기층용, 중간층용 - 안정도'],
 '기층용, 중간층용 - 체가름': ['기층용, 중간층용 - 체가름'],
 '기층용, 중간층용,표층용 - 입도': ['기층용, 중간층용,표층용 - 입도'],
 '기층용,중간층용 - 간접인장강도': ['기층용,중간층용 - 간접인장강도'],
 '기층용,중간층용 - 공극률': ['기층용,중간층용 - 공극률'],
 '기층용,중간층용 - 다짐회수': ['기층용,중간층용 - 다짐회수'],
 '기층용,중간층용 - 변형강도': ['기층용,중간층용 - 변형강도'],
 '기층용,중간층용 - 포화도': ['기층용,중간층용 - 포화도'],
 '기층용,중간층용 - 흐름값': ['기층용,중간층용 - 흐름값'],
 '기층용,중간층용,표층용 - 아스팔트함량': ['기층용,중간층용,표층용 - 아스팔트함량', '기층용,중간층용 - 아스팔트 함유량'],
 '기포 슬러리 밀도': ['기포 슬러리 밀도'],
 '기포간격계수': ['기포간격계수'],
 '기포율': ['기포율'],
 '기포제': ['기포제'],
 '길이': ['길이'],
 '길이변화비': ['길이변화비'],
 '길이변화율': ['길이변화율'],
 '길이선열 팽창계수': ['길이선열 팽창계수'],
 '꺽임 강도': ['꺽임 강도'],
 '꺾임강도': ['꺾임강도'],
 '꼬임 길이': ['꼬임 길이'],
 '꿰뚫림강도': ['꿰뚫림강도'],
 '나사': ['나사'],
 '나사못 유지력': ['나사못 유지력'],
 '낙구 충격 파괴 강도': ['낙구 충격 파괴 강도'],
 '낙구충격시험': ['낙구충격시험'],
 '낙하충격 성능': ['낙하충격 성능'],
 '난연성': ['난연성', '난연성능'],
 '납': ['납'],
 '내?외경허용차': ['내?외경허용차'],
 '내경 5% 변형 시 강성': ['내경 5% 변형 시 강성'],
 '내곰팡이성': ['내곰팡이성'],
 '내구성': ['내구성'],
 '내균열성': ['내균열성', '내열성', '내열성능', '내열수성'],
 '내긁힘성': ['내긁힘성'],
 '내동해성': ['내동해성'],
 '내력': ['내력'],
 '내마모성': ['내마모성', '마모성'],
 '내마모성, 내알칼리성': ['내마모성, 내알칼리성'],
 '내변퇴색성': ['내변퇴색성'],
 '내부마찰각': ['내부마찰각'],
 '내산성': ['내산성'],
 '내세척성': ['내세척성'],
 '내수성': ['내수성', '내투수성', '내수압성'],
 '내수성, 내알칼리성': ['내수성, 내알칼리성'],
 '내시너성': ['내시너성'],
 '내알카리성': ['내알카리성'],
 '내알칼리성': ['내알칼리성'],
 '내약품성': ['내약품성'],
 '내약품성℃, 6h)후 외관': ['내약품성℃, 6h)후 외관'],
 '내연성': ['내연성'],
 '내열 강성 변화율': ['내열 강성 변화율'],
 '내열 치수 안정성': ['내열 치수 안정성'],
 '내움푹 패임 성능': ['내움푹 패임 성능'],
 '내잔갈림성': ['내잔갈림성'],
 '내충격성': ['내충격성'],
 '내투수성능': ['내투수성능'],
 '내피로성능': ['내피로성능', '내피로성'],
 '내하중성': ['내하중성'],
 '내한성': ['내한성'],
 '내화성': ['내화성', '내황화성'],
 '내화염성': ['내화염성'],
 '내화염성, 연소성능': ['내화염성, 연소성능'],
 '내화학 성능 후 인장성능': ['내화학 성능 후 인장성능'],
 '내화학성': ['내화학성'],
 '내후도': ['내후도'],
 '내후성': ['내후성'],
 '내휘발유성, 나이프시험': ['내휘발유성, 나이프시험'],
 '내흡수 성능': ['내흡수 성능'],
 '냉동 안정성': ['냉동 안정성'],
 '냉열반복시험': ['냉열반복시험'],
 '너비': ['너비'],
 '노상토지지력비': ['노상토지지력비'],
 '노치민감도비율': ['노치민감도비율'],
 '노화 전 연신율': ['노화 전 연신율'],
 '노화 후 연신율': ['노화 후 연신율'],
 '노화 후 인장강도': ['노화 후 인장강도', '노화 전 인장강도'],
 '노화성': ['노화성'],
 '노화시험': ['노화시험'],
 '녹': ['녹'],
 '높이': ['높이'],
 '니켈': ['니켈'],
 '다짐': ['다짐'],
 '다짐밀도': ['다짐밀도'],
 '다짐횟수': ['다짐횟수'],
 '단면 경화층의 경도 분포': ['단면 경화층의 경도 분포'],
 '단면수축률': ['단면수축률'],
 '단면적': ['단면적'],
 '단열성': ['단열성'],
 '단열성 시험': ['단열성 시험'],
 '단위길이당 무게': ['단위길이당 무게'],
 '단위면적당 질량': ['단위면적당 질량'],
 '단위무게': ['단위무게'],
 '단위수량': ['단위수량'],
 '단위용적잘량': ['단위용적잘량'],
 '단위용적중량': ['단위용적중량'],
 '단위용적질량': ['단위용적질량', '습윤단위용적질량'],
 '단위중량': ['단위중량'],
 '도금부착량': ['도금부착량', '도금의 부착량'],
 '도로평판': ['도로평판'],
 '도막 두께': ['도막 두께'],
 '도막 밀착력': ['도막 밀착력'],
 '도막, 피복재와 바탕합판의 접착성': ['도막, 피복재와 바탕합판의 접착성'],
 '도막부착성': ['도막부착성'],
 '도막의 겉모양': ['도막의 겉모양'],
 '도막의 상태': ['도막의 상태'],
 '도포 작업성, 지촉 건조 시간': ['도포 작업성, 지촉 건조 시간'],
 '돌출': ['돌출'],
 '동결 융해': ['동결 융해'],
 '동결 융해에 대한 저항성': ['동결 융해에 대한 저항성'],
 '동결-해동 반복에 따른 안정도 시험 후 응고분': ['동결-해동 반복에 따른 안정도 시험 후 응고분'],
 '동결박리 저항성': ['동결박리 저항성'],
 '동결융해 저항성': ['동결융해 저항성'],
 '동재하': ['동재하'],
 '동적마찰계수': ['동적마찰계수'],
 '동적수침 후 피복률': ['동적수침 후 피복률'],
 '동적안정도': ['동적안정도'],
 '동탄성계수': ['동탄성계수'],
 '두께': ['두께'],
 '둥근 모': ['둥근 모'],
 '뒤틀림': ['뒤틀림'],
 '드레인 다운': ['드레인 다운'],
 '드레인재': ['드레인재'],
 '등가알칼리량': ['등가알칼리량'],
 '등급': ['등급'],
 '등록된 자료 없음': ['등록된 자료 없음'],
 '록심부 인장강도': ['록심부 인장강도'],
 '류엘유동성': ['류엘유동성'],
 '리브높이': ['리브높이'],
 '리프트오프시험': ['리프트오프시험'],
 '릴랙세이션': ['릴랙세이션'],
 '마모': ['마모'],
 '마모 저항성': ['마모 저항성'],
 '마모감량': ['마모감량'],
 '마모율': ['마모율'],
 '마샬안정도': ['마샬안정도'],
 '마찰견뢰도': ['마찰견뢰도'],
 '마크로부식시험': ['마크로부식시험'],
 '마크로시험': ['마크로시험'],
 '만곡': ['만곡'],
 '맞이음의 내부 결함': ['맞이음의 내부 결함'],
 '모래당량': ['모래당량'],
 '모래당량시험': ['모래당량시험'],
 '모래의 최대크기': ['모래의 최대크기'],
 '모래의 함량': ['모래의 함량'],
 '모르타르의 압축강도비': ['모르타르의 압축강도비', '모르타르 압축강도비'],
 '모스 경도': ['모스 경도'],
 '모양': ['모양'],
 '모양, 치수, 무게 및 그 허용차': ['모양, 치수, 무게 및 그 허용차'],
 '모양, 치수, 이형 봉강 1개의 무게 허용차': ['모양, 치수, 이형 봉강 1개의 무게 허용차'],
 '모양, 치수, 질량': ['모양, 치수, 질량', '모양, 치수, 단위질량'],
 '모인 옹이': ['모인 옹이'],
 '몬모릴로나이트함량': ['몬모릴로나이트함량'],
 '무게': ['무게'],
 '무게변화': ['무게변화'],
 '물에 녹지 않는 물질': ['물에 녹지 않는 물질'],
 '물흡수 계수': ['물흡수 계수', '물흡수 계수비'],
 '미끄러짐에 의한 변화량': ['미끄러짐에 의한 변화량'],
 '미끄럼 저항감소': ['미끄럼 저항감소'],
 '미끄럼저항': ['미끄럼저항', '미끄럼 저항성', '미끄럼 저항'],
 '미끄럼저항계수': ['미끄럼저항계수'],
 '미끄럼저항시험': ['미끄럼저항시험'],
 '밀도': ['밀도'],
 '밀도 2.0g/㎤의 액체에 뜨는것': ['밀도 2.0g/㎤의 액체에 뜨는것'],
 '밀도, 함수율': ['밀도, 함수율'],
 '밀도, 흡수율': ['밀도, 흡수율', '밀도 및 흡수율'],
 '밀도의 차': ['밀도의 차'],
 '밀도편차': ['밀도편차'],
 '밀착성': ['밀착성'],
 '밀착성 시험': ['밀착성 시험'],
 '바닥재의 처짐': ['바닥재의 처짐'],
 '바닥재의 처짐 및 휨시험': ['바닥재의 처짐 및 휨시험'],
 '박리 강도': ['박리 강도', '박리강도'],
 '박리 시험': ['박리 시험'],
 '박리 저항성': ['박리 저항성'],
 '박리량': ['박리량'],
 '반복 침지 건조 시험': ['반복 침지 건조 시험'],
 '방사선투과시험': ['방사선투과시험'],
 '방습코팅': ['방습코팅'],
 '방염성능': ['방염성능'],
 '방염성능시험': ['방염성능시험'],
 '방청성능': ['방청성능', '방청성'],
 '배수능력': ['배수능력'],
 '배수성능': ['배수성능'],
 '배합설계': ['배합설계'],
 '벤젠': ['벤젠'],
 '벤조피렌': ['벤조피렌'],
 '변색시험': ['변색시험'],
 '변위량': ['변위량'],
 '변위량 측정': ['변위량 측정'],
 '변의길이': ['변의길이'],
 '변형강도': ['변형강도'],
 '보수성': ['보수성'],
 '보수율': ['보수율'],
 '보증하중': ['보증하중'],
 '보증하중응력': ['보증하중응력'],
 '봉수깊이': ['봉수깊이'],
 '봉합강도': ['봉합강도'],
 '부유물질': ['부유물질'],
 '부재의모양 안정성': ['부재의모양 안정성', '부재모양안정성'],
 '부착 성능': ['부착 성능', '부착성'],
 '부착강도': ['부착강도', '부착 강도', '부착도'],
 '부착강도 시험 후 외관': ['부착강도 시험 후 외관'],
 '부착성능-냉온반복처리후': ['부착성능-냉온반복처리후'],
 '부착성능-냉온반복처리후 도막상태': ['부착성능-냉온반복처리후 도막상태'],
 '부착성능-무처리': ['부착성능-무처리'],
 '부피 비중': ['부피 비중'],
 '부피고형분': ['부피고형분'],
 '부피변화': ['부피변화'],
 '부피손실': ['부피손실'],
 '분말도': ['분말도'],
 '분진량': ['분진량'],
 '분할': ['분할'],
 '불소': ['불소'],
 '불연성 시험': ['불연성 시험'],
 '불점착 건조성': ['불점착 건조성'],
 '불휘발분': ['불휘발분'],
 '브리넬 경도-지름 20 초과': ['브리넬 경도-지름 20 초과'],
 '블리딩률': ['블리딩률'],
 '블리딩률, 팽창률': ['블리딩률, 팽창률'],
 '블리딩성': ['블리딩성'],
 '블리딩양의 비': ['블리딩양의 비'],
 '블리딩의 차': ['블리딩의 차'],
 '비소': ['비소'],
 '비중': ['비중'],
 '비중 및 흡수율': ['비중 및 흡수율'],
 '비카트 연화온도': ['비카트 연화온도'],
 '비탈탄 나사부의 최소 높이, E': ['비탈탄 나사부의 최소 높이, E'],
 '비틀림': ['비틀림'],
 '비틀림 강도': ['비틀림 강도'],
 '비틀림 특성': ['비틀림 특성'],
 '비틀림시험': ['비틀림시험'],
 '비파괴 검사특성 또는 수압시험특성': ['비파괴 검사특성 또는 수압시험특성'],
 '비파괴 시험': ['비파괴 시험'],
 '비표면적': ['비표면적'],
 '비휘발 전색제': ['비휘발 전색제', '비휘발 전색제분'],
 '비휘발분': ['비휘발분'],
 '비휘발성 물질': ['비휘발성 물질'],
 '사이클부식성': ['사이클부식성'],
 '산값': ['산값'],
 '산화 소듐': ['산화 소듐'],
 '산화 철': ['산화 철'],
 '산화마그네슘': ['산화마그네슘'],
 '산화마그네슘, 강열감량': ['산화마그네슘, 강열감량'],
 '삼산화황': ['삼산화황'],
 '삼산화황 정량방법': ['삼산화황 정량방법'],
 '상도 적합성': ['상도 적합성'],
 '상태': ['상태'],
 '색도좌표': ['색도좌표'],
 '색상': ['색상'],
 '색차': ['색차'],
 '샤르피 흡수에너지': ['샤르피 흡수에너지'],
 '샤르피충격값': ['샤르피충격값'],
 '샤르피충격강도': ['샤르피충격강도'],
 '샤르피충격시험': ['샤르피충격시험'],
 '석면': ['석면'],
 '석유계총탄화수소': ['석유계총탄화수소'],
 '석탄, 갈탄 함유량': ['석탄, 갈탄 함유량', '석탄 및 갈탄 함유량'],
 '선지름': ['선지름', '소선지름'],
 '섬유 함유량': ['섬유 함유량'],
 '섬유층 분리강도': ['섬유층 분리강도'],
 '성분시험': ['성분시험'],
 '세로': ['세로'],
 '세립토 비율': ['세립토 비율'],
 '세츄레이트': ['세츄레이트'],
 '소성변형률': ['소성변형률'],
 '소성연신율': ['소성연신율'],
 '소성지수': ['소성지수'],
 '소성한계': ['소성한계'],
 '손실계수': ['손실계수'],
 '쇄석최대치수': ['쇄석최대치수'],
 '쇼어경도': ['쇼어경도'],
 '쇼트백 충격 특성': ['쇼트백 충격 특성'],
 '수밀성능': ['수밀성능', '수밀 성능', '수밀성'],
 '수분': ['수분'],
 '수분 함량': ['수분 함량'],
 '수분율': ['수분율'],
 '수소 이온 농도': ['수소 이온 농도'],
 '수은': ['수은'],
 '수정CBR': ['수정CBR'],
 '수중 침적 후의 일정 신장 하에서의 접착성': ['수중 침적 후의 일정 신장 하에서의 접착성'],
 '수중 침적후의 접착성파괴시의 신장률': ['수중 침적후의 접착성파괴시의 신장률'],
 '수중단위용적질량': ['수중단위용적질량'],
 '수증기투과도': ['수증기투과도'],
 '수직 처짐량': ['수직 처짐량'],
 '수직/수평 시험': ['수직/수평 시험'],
 '수직압축하중': ['수직압축하중'],
 '수직투수계수': ['수직투수계수', '수직 투수계수'],
 '수축신장시험': ['수축신장시험'],
 '수침 7일 후의 인장 접착 강도 시험': ['수침 7일 후의 인장 접착 강도 시험'],
 '수침잔류안정도': ['수침잔류안정도'],
 '수침팽창비': ['수침팽창비'],
 '수평배수성능': ['수평배수성능'],
 '수평재하': ['수평재하'],
 '수평재하시험': ['수평재하시험'],
 '수평지지력': ['수평지지력'],
 '수평투수계수': ['수평투수계수'],
 '수화열': ['수화열'],
 '슈미트해머': ['슈미트해머'],
 '스프레이 작업성': ['스프레이 작업성'],
 '슬러지수의 농도': ['슬러지수의 농도'],
 '슬럼프': ['슬럼프'],
 '슬럼프 또는 슬럼프플로': ['슬럼프 또는 슬럼프플로'],
 '슬럼프 증가량': ['슬럼프 증가량'],
 '습기유지': ['습기유지'],
 '습기투과 저항성': ['습기투과 저항성'],
 '습열성': ['습열성'],
 '습윤강도': ['습윤강도'],
 '습윤시 휨 강도': ['습윤시 휨 강도'],
 '시공성': ['시공성'],
 '시멘트 응결시간의 차': ['시멘트 응결시간의 차'],
 '시멘트 혼화용 폴리머 고형분': ['시멘트 혼화용 폴리머 고형분'],
 '시안': ['시안'],
 '시험 속도 : 100 mm/min 지점 간 거리 : 192 mm': ['시험 속도 : 100 mm/min 지점 간 거리 : 192 mm'],
 '시험일자 : 2025. 06. 06 설계하중 : 2,600 kN/본': ['시험일자 : 2025. 06. 06 설계하중 : 2,600 kN/본'],
 '신도': ['신도'],
 '신율': ['신율'],
 '신장, 수축': ['신장, 수축'],
 '신장률': ['신장률'],
 '신장시 열화 성상': ['신장시 열화 성상', '신장시의 열화성상'],
 '실내공기 오염물질 방출량': ['실내공기 오염물질 방출량'],
 '실내다짐밀도': ['실내다짐밀도'],
 '실내투수계수': ['실내투수계수'],
 '실적율': ['실적율'],
 '심부경도': ['심부경도'],
 '심선 지름과 측선 지름의 차': ['심선 지름과 측선 지름의 차'],
 '썩음': ['썩음'],
 '쐐기인장하중': ['쐐기인장하중'],
 '씻기시험에서 손실되는 양': ['씻기시험에서 손실되는 양'],
 '아스팔트 추출 시험': ['아스팔트 추출 시험'],
 '아스팔트 함량': ['아스팔트 함량', '구재 아스팔트 함량'],
 '아스팔트 함유량': ['아스팔트 함유량'],
 '아연': ['아연'],
 '아연 부착량': ['아연 부착량'],
 '아연도금 부착량': ['아연도금 부착량'],
 '아이죠드충격강도': ['아이죠드충격강도'],
 '안료분': ['안료분'],
 '안정도': ['안정도'],
 '안정성': ['안정성', '열안정성'],
 '알칼리 골재 반응': ['알칼리 골재 반응', '알칼리 골재 반응 시험', '알칼리 골재 반응 시험*'],
 '알칼리농도감소량': ['알칼리농도감소량'],
 '알칼리잠재반응결과분석': ['알칼리잠재반응결과분석'],
 '압밀배수조건하의 직접전단시험': ['압밀배수조건하의 직접전단시험'],
 '압입량': ['압입량'],
 '압축 가열 · 인장 냉각후의 접착성': ['압축 가열 · 인장 냉각후의 접착성'],
 '압축 강도': ['압축 강도', '압축강도', '압축 강도비'],
 '압축 시험': ['압축 시험'],
 '압축 하중': ['압축 하중', '압축하중'],
 '압축강도 28일': ['압축강도 28일'],
 '압축강도 7일': ['압축강도 7일'],
 '압축강도 및 흡수율': ['압축강도 및 흡수율'],
 '압축강도 시험': ['압축강도 시험', '일축압축강도시험'],
 '압축강성': ['압축강성'],
 '압축경도': ['압축경도'],
 '압축변형하중': ['압축변형하중'],
 '압축영구 변형율': ['압축영구 변형율'],
 '압축영구 줄음율 시험': ['압축영구 줄음율 시험'],
 '압축응력': ['압축응력', '압축력'],
 '압축크리프': ['압축크리프'],
 '압축파괴하중': ['압축파괴하중'],
 '액비중': ['액비중'],
 '액성한계': ['액성한계'],
 '액성한계·소성한계': ['액성한계·소성한계'],
 '액체저항성': ['액체저항성'],
 '앵글러도': ['앵글러도'],
 '양방향 재하시험': ['양방향 재하시험', '횡방향재하시험'],
 '양이온치환용량': ['양이온치환용량'],
 '에틸벤젠': ['에틸벤젠'],
 '역청함유량': ['역청함유량'],
 '연도변화': ['연도변화'],
 '연석량': ['연석량'],
 '연소성': ['연소성', '연소성능', '내연소성'],
 '연신율': ['연신율'],
 '연신율 판-두께 20 mm 이하': ['연신율 판-두께 20 mm 이하'],
 '연직 하중 강도': ['연직 하중 강도'],
 '연직도시험': ['연직도시험'],
 '연한 석편': ['연한 석편'],
 '연화도': ['연화도'],
 '연화점': ['연화점'],
 '열 관류 저항': ['열 관류 저항'],
 '열 호환성': ['열 호환성'],
 '열간 수축 온도': ['열간 수축 온도'],
 '열간내압크리프성': ['열간내압크리프성'],
 '열관류율': ['열관류율'],
 '열방출 시험': ['열방출 시험'],
 '열저항': ['열저항', '인열저항'],
 '열전도도': ['열전도도'],
 '열전도율': ['열전도율'],
 '열특성': ['열특성'],
 '열팽창': ['열팽창'],
 '열팽창계수': ['열팽창계수'],
 '열화 처리 후 인장성능': ['열화 처리 후 인장성능'],
 '열화처리후의 인장강도비': ['열화처리후의 인장강도비', '열화처리후 인장강도비'],
 '열화처리후의 파단시 신장률': ['열화처리후의 파단시 신장률'],
 '염분농도': ['염분농도'],
 '염분침투 저항성': ['염분침투 저항성'],
 '염소 이온 침투 저항성': ['염소 이온 침투 저항성'],
 '염소이온 침투깊이': ['염소이온 침투깊이'],
 '염소이온량': ['염소이온량', '염소이온'],
 '염수분무': ['염수분무'],
 '염화 이온 침투': ['염화 이온 침투'],
 '염화 이온 침투 저항성': ['염화 이온 침투 저항성', '염화물 이온 침투 저항성', '염화 이온 침투 저항성능'],
 '염화물 이온': ['염화물 이온'],
 '염화물 함유량': ['염화물 함유량'],
 '염화물량': ['염화물량', '염화물', '염화물함량'],
 '염화물시험': ['염화물시험'],
 '영구 신장률': ['영구 신장률'],
 '영구 압축 줄음률': ['영구 압축 줄음률', '영구압축줄음율'],
 '영구잔류변위량': ['영구잔류변위량'],
 '오염성': ['오염성', '내오염성'],
 '오존균열 시험': ['오존균열 시험', '오존균열시험'],
 '온도': ['온도'],
 '온도 의존성': ['온도 의존성'],
 '온도의존성-인장강도비': ['온도의존성-인장강도비'],
 '온도의존성-파단시 신장률': ['온도의존성-파단시 신장률'],
 '옹이': ['옹이'],
 '옹이 지름비': ['옹이 지름비'],
 '완전 탈탄부의 최대 높이,G': ['완전 탈탄부의 최대 높이,G'],
 '외관 및 치수검사': ['외관 및 치수검사'],
 '외관검사': ['외관검사'],
 '외관상태, 성능': ['외관상태, 성능'],
 '외압강도': ['외압강도'],
 '용기내 상태': ['용기내 상태'],
 '용기내에서의 상태': ['용기내에서의 상태', '용기내의 상태'],
 '용융지수': ['용융지수'],
 '용접부 인장강도': ['용접부 인장강도'],
 '용접부 인장강도- 외경 350 초과': ['용접부 인장강도- 외경 350 초과'],
 '용접부 인장하중': ['용접부 인장하중', '접합부 인장하중'],
 '용접부의 내부결함': ['용접부의 내부결함'],
 '용접부의 비파괴검사': ['용접부의 비파괴검사', '용접부 비파괴검사'],
 '용접비이드의 외관, 형상': ['용접비이드의 외관, 형상'],
 '용접사 기량테스트비파괴검사': ['용접사 기량테스트비파괴검사'],
 '용접점 전단강도': ['용접점 전단강도'],
 '용접점의 박리': ['용접점의 박리'],
 '용착금속 인장강도': ['용착금속 인장강도'],
 '용해 실리카량': ['용해 실리카량', '용해실리카'],
 '용해성 증발 잔류물의 양': ['용해성 증발 잔류물의 양', '용해성 증발 잔유물의 양'],
 '원강성': ['원강성'],
 '원연성': ['원연성'],
 '원연성 시험': ['원연성 시험'],
 '위치': ['위치'],
 '위치교정도': ['위치교정도'],
 '유기 불순물': ['유기 불순물'],
 '유기 이물질함유량': ['유기 이물질함유량', '무기 이물질함유량', '이물질 함유량'],
 '유기물 함량': ['유기물 함량'],
 '유기인': ['유기인'],
 '유기인화합물': ['유기인화합물'],
 '유동성': ['유동성'],
 '유동점': ['유동점'],
 '유리섬유함유량': ['유리섬유함유량'],
 '유리알 겉모양, 모양': ['유리알 겉모양, 모양'],
 '유리알 함유량': ['유리알 함유량', '유리함유량'],
 '유색층 두께': ['유색층 두께', '유색층의 두께', '유색층두께'],
 '유연온도': ['유연온도'],
 '유체손실율': ['유체손실율'],
 '유하시간': ['유하시간'],
 '유해물질': ['유해물질'],
 '유효구멍크기': ['유효구멍크기'],
 '유효인산': ['유효인산'],
 '육안검사': ['육안검사'],
 '윤할': ['윤할'],
 '은폐성': ['은폐성'],
 '은폐율': ['은폐율'],
 '음향감쇠계수': ['음향감쇠계수'],
 '응결': ['응결'],
 '응결 시간': ['응결 시간', '응결 시간 차'],
 '응고분': ['응고분'],
 '이론최대밀도': ['이론최대밀도'],
 '이물질함량': ['이물질함량'],
 '이물질함유량/무기이물질': ['이물질함유량/무기이물질', '이물질함유량/유기이물질'],
 '이산화규소': ['이산화규소'],
 '이슬점': ['이슬점'],
 '이음매단차': ['이음매단차'],
 '이음인장강도': ['이음인장강도'],
 '인공광 노출 후의 접착성': ['인공광 노출 후의 접착성'],
 '인발부착강도': ['인발부착강도'],
 '인발시험': ['인발시험'],
 '인발재하': ['인발재하'],
 '인발재하시험': ['인발재하시험'],
 '인열 성능': ['인열 성능'],
 '인열강도': ['인열강도'],
 '인열성능-인열강도': ['인열성능-인열강도'],
 '인열시험': ['인열시험'],
 '인장': ['인장'],
 '인장 내구성': ['인장 내구성'],
 '인장 접착 강도': ['인장 접착 강도'],
 '인장 최대하중': ['인장 최대하중'],
 '인장강도': ['인장강도', '휨인장강도', '인장강도비'],
 '인장강도 판-두께 20 mm 이하': ['인장강도 판-두께 20 mm 이하'],
 '인장강도, 신도': ['인장강도, 신도'],
 '인장강도, 연신율': ['인장강도, 연신율'],
 '인장강도, 인장변형률': ['인장강도, 인장변형률', '인장강도, 인장변형'],
 '인장변형률': ['인장변형률'],
 '인장성능': ['인장성능'],
 '인장시험': ['인장시험'],
 '인장신도': ['인장신도', '인장강신도'],
 '인장신율': ['인장신율'],
 '인장응력': ['인장응력', '인장력'],
 '인장전단': ['인장전단'],
 '인장전단접착강도': ['인장전단접착강도', '전단접착강도', '경사전단 접착강도'],
 '인장탄성율': ['인장탄성율'],
 '인장파괴시 신장률': ['인장파괴시 신장률'],
 '인장파단변형': ['인장파단변형'],
 '인장파단응력': ['인장파단응력'],
 '인장하중': ['인장하중'],
 '인장항복 강도': ['인장항복 강도'],
 '인화점': ['인화점'],
 '일 방향 인장시험': ['일 방향 인장시험'],
 '일광견뢰도': ['일광견뢰도'],
 '일정 신장 하에서의 접착성': ['일정 신장 하에서의 접착성'],
 '일정신장시 인장강도': ['일정신장시 인장강도'],
 '입경': ['입경'],
 '입도': ['입도'],
 '입도 시험': ['입도 시험'],
 '입도, 조립률': ['입도, 조립률', '입도 및 조립률'],
 '입자모양판정 실적률': ['입자모양판정 실적률'],
 '입자의 전하': ['입자의 전하'],
 '자기비파괴검사': ['자기비파괴검사'],
 '자분탐상': ['자분탐상'],
 '자분탐상검사': ['자분탐상검사'],
 '자분탐상시험': ['자분탐상시험'],
 '작업성': ['작업성', '붓 작업성'],
 '작업시간': ['작업시간'],
 '잔골재 공극률': ['잔골재 공극률'],
 '잔골재의 표면수량': ['잔골재의 표면수량'],
 '잔류변형량': ['잔류변형량', '잔류변형'],
 '잔류압입률': ['잔류압입률'],
 '잔염시간': ['잔염시간'],
 '잔진시간': ['잔진시간'],
 '장기 열전도율': ['장기 열전도율'],
 '장기열저항': ['장기열저항'],
 '재귀반사성능시험': ['재귀반사성능시험', '재귀반사성능'],
 '재질': ['재질'],
 '재질시험': ['재질시험'],
 '재템퍼링 후 경도차': ['재템퍼링 후 경도차'],
 '재하 강도': ['재하 강도'],
 '재하시험': ['재하시험', '정재하시험'],
 '저 사이클 반복시험': ['저 사이클 반복시험', '저사이클반복시험', '고사이클반복시험'],
 '저온 굴곡성': ['저온 굴곡성'],
 '저온 안정성': ['저온 안정성'],
 '저온작업성': ['저온작업성'],
 '저장 안정성': ['저장 안정성', '저장안정성능'],
 '저장성': ['저장성'],
 '저장안정도': ['저장안정도'],
 '적신 도막 은폐율': ['적신 도막 은폐율'],
 '적외선 흡수스펙트럼': ['적외선 흡수스펙트럼'],
 '전기고유저항': ['전기고유저항'],
 '전기전도도': ['전기전도도'],
 '전단 접착 성능': ['전단 접착 성능'],
 '전단강도': ['전단강도'],
 '전단강성/EDC': ['전단강성/EDC'],
 '전단계수': ['전단계수'],
 '전단압축하중': ['전단압축하중'],
 '전단유효강성': ['전단유효강성'],
 '전체 알칼리량': ['전체 알칼리량'],
 '전체 처짐량': ['전체 처짐량'],
 '절건 밀도': ['절건 밀도'],
 '절건비중': ['절건비중'],
 '절대 건조 밀도, 흡수율': ['절대 건조 밀도, 흡수율', '절대 건조 밀도 및 흡수율', '절대건조건밀도 및 흡수율'],
 '절대점도': ['절대점도'],
 '절연저항': ['절연저항'],
 '점도': ['점도'],
 '점성': ['점성'],
 '점적 시험': ['점적 시험'],
 '점착력': ['점착력'],
 '점토덩어리 함유랑': ['점토덩어리 함유랑'],
 '점토덩어리 함유량': ['점토덩어리 함유량', '점토덩어리', '점토덩어리량'],
 '점하중시험': ['점하중시험'],
 '접착강도': ['접착강도', '접착 강도'],
 '접착성': ['접착성'],
 '접합 강도': ['접합 강도'],
 '접합 안정 성능': ['접합 안정 성능'],
 '접합성능': ['접합성능'],
 '접합성상': ['접합성상'],
 '정량': ['정량'],
 '정재하': ['정재하'],
 '정적내력시험': ['정적내력시험'],
 '정탄성계수': ['정탄성계수', '탄성계수'],
 '정하중': ['정하중'],
 '정하중시험': ['정하중시험'],
 '정하중시험 [뚜껑의 지름': ['정하중시험 [뚜껑의 지름'],
 '제품인장시험': ['제품인장시험'],
 '조립률 포함': ['조립률 포함'],
 '조성섬유': ['조성섬유'],
 '종방향': ['종방향'],
 '주도': ['주도'],
 '주사전자현미경을 이용한 EDS시험': ['주사전자현미경을 이용한 EDS시험'],
 '준불연성': ['준불연성'],
 '줄눈너비의 신장율': ['줄눈너비의 신장율'],
 '중량': ['중량'],
 '중성염수분무시험': ['중성염수분무시험', '염수분무시험'],
 '중성화 저항성': ['중성화 저항성'],
 '증류 시험': ['증류 시험'],
 '증류시험{360 ℃까지의 시료에 대한 부피}': ['증류시험{360 ℃까지의 시료에 대한 부피}'],
 '증류찌끼시험-침입도': ['증류찌끼시험-침입도'],
 '증류찌끼시험-톨루엔에 녹는 물질': ['증류찌끼시험-톨루엔에 녹는 물질'],
 '증발 잔류물': ['증발 잔류물'],
 '증발잔류분 질량': ['증발잔류분 질량'],
 '지름': ['지름'],
 '지름, 최소피막두께': ['지름, 최소피막두께'],
 '지지대의 휨 하중': ['지지대의 휨 하중'],
 '지지력': ['지지력'],
 '지촉건조': ['지촉건조'],
 '지촉건조시간': ['지촉건조시간'],
 '지하수전항목 의뢰시': ['지하수전항목 의뢰시'],
 '직각도': ['직각도'],
 '직경': ['직경'],
 '직접전단': ['직접전단'],
 '질량': ['질량'],
 '질량 변화율': ['질량 변화율'],
 '질량손실': ['질량손실'],
 '집수공 너비': ['집수공 너비'],
 '집중하중안전성': ['집중하중안전성'],
 '차연성': ['차연성'],
 '채움재의다짐공극률': ['채움재의다짐공극률'],
 '철근삽입하중': ['철근삽입하중'],
 '철망의 크기': ['철망의 크기'],
 '철분함유량': ['철분함유량'],
 '체 불통과분': ['체 불통과분'],
 '체가름': ['체가름'],
 '체결 축력': ['체결 축력'],
 '체잔류분 질량': ['체잔류분 질량'],
 '초기 열전도도': ['초기 열전도도'],
 '초기 열전도율': ['초기 열전도율'],
 '초음파 탐상검사': ['초음파 탐상검사', '초음파탐사검사'],
 '초음파비파괴검사': ['초음파비파괴검사'],
 '초음파탐상시험': ['초음파탐상시험'],
 '촉진 노화 시험': ['촉진 노화 시험'],
 '촉진내후성': ['촉진내후성'],
 '촉진내후성시험후 광택유지율': ['촉진내후성시험후 광택유지율'],
 '촉진내후성시험후 외관, 색차': ['촉진내후성시험후 외관, 색차', '촉진내후성시험후 외관'],
 '촉진내후성시험후 황변도, 흐림도변화': ['촉진내후성시험후 황변도, 흐림도변화'],
 '총 알칼리': ['총 알칼리'],
 '총 인': ['총 인'],
 '총 질소': ['총 질소'],
 '총아연말분': ['총아연말분'],
 '총휘발성유기화합물, 톨루엔, 폼알데하이드': ['총휘발성유기화합물, 톨루엔, 폼알데하이드'],
 '최대건조밀도': ['최대건조밀도'],
 '최대치수': ['최대치수'],
 '최대하중': ['최대하중', '최대 휨 하중'],
 '최소 인장하중': ['최소 인장하중'],
 '최소도금부착량': ['최소도금부착량'],
 '최소두께': ['최소두께'],
 '최적 함수비': ['최적 함수비'],
 '추출 입도': ['추출 입도'],
 '추출AP함유량': ['추출AP함유량'],
 '출석률': ['출석률'],
 '충격 강도': ['충격 강도'],
 '충격시험': ['충격시험'],
 '층간부착성': ['층간부착성'],
 '치수': ['치수'],
 '치수 및 모양': ['치수 및 모양'],
 '치수, 단위무게': ['치수, 단위무게'],
 '치수, 무게': ['치수, 무게'],
 '치수, 제품의 단위면적무게': ['치수, 제품의 단위면적무게'],
 '치수변화율': ['치수변화율'],
 '치수안정성': ['치수안정성'],
 '치수안정성 ℃, 48 h)': ['치수안정성 ℃, 48 h)'],
 '치수오차': ['치수오차'],
 '치수의 불규칙도': ['치수의 불규칙도'],
 '치환성양이온': ['치환성양이온'],
 '침수팽창': ['침수팽창'],
 '침입도': ['침입도'],
 '침입도 잔류율': ['침입도 잔류율'],
 '침입도의 비': ['침입도의 비'],
 '침입방어성능': ['침입방어성능'],
 '침지박리': ['침지박리'],
 '침지성': ['침지성'],
 '침지시험': ['침지시험'],
 '침투 깊이': ['침투 깊이'],
 '침하깊이': ['침하깊이'],
 '카드뮴': ['카드뮴', '납,카드뮴'],
 '카본블랙함량': ['카본블랙함량'],
 '칸타브로': ['칸타브로'],
 '칸타브로 손실률': ['칸타브로 손실률'],
 '컨시스턴시': ['컨시스턴시'],
 '코아채취': ['코아채취'],
 '콘크리트 박리상태': ['콘크리트 박리상태'],
 '콘크리트 폼상태': ['콘크리트 폼상태'],
 '콘크리트 표면상태': ['콘크리트 표면상태'],
 '콘크리트의 압축강도비': ['콘크리트의 압축강도비'],
 '크실렌': ['크실렌'],
 '타이어 부착 손실량': ['타이어 부착 손실량'],
 '탁도': ['탁도'],
 '탄성변위량': ['탄성변위량'],
 '탄성복원성': ['탄성복원성'],
 '탄성파 속도': ['탄성파 속도'],
 '탄성회복률': ['탄성회복률'],
 '탄소당량': ['탄소당량'],
 '탄소당량 또는 용접 균열 감수성': ['탄소당량 또는 용접 균열 감수성', '탄소당량 또는 용접 균열 감수성 조성'],
 '탄화길이': ['탄화길이'],
 '탄화면적': ['탄화면적'],
 '태양열 제거율': ['태양열 제거율'],
 '터프니스': ['터프니스'],
 '테이버마모시험': ['테이버마모시험'],
 '테트라클로로에틸렌': ['테트라클로로에틸렌'],
 '토양산도': ['토양산도'],
 '토질조사': ['토질조사'],
 '토크계수값': ['토크계수값'],
 '톨루엔': ['톨루엔'],
 '톨루엔 가용분 질량': ['톨루엔 가용분 질량', '톨루엔 가용분'],
 '투과손실 시험 시료사이즈 3,570 × 2,750 mm': ['투과손실 시험 시료사이즈 3,570 × 2,750 mm'],
 '투과손실시험': ['투과손실시험'],
 '투수': ['투수'],
 '투수계수': ['투수계수'],
 '투수량': ['투수량'],
 '투수비': ['투수비'],
 '투수성': ['투수성'],
 '투수시험': ['투수시험'],
 '투수지속성 시험': ['투수지속성 시험'],
 '투습계수': ['투습계수'],
 '트리클로로에탄 가용분': ['트리클로로에탄 가용분'],
 '트리클로로에틸렌': ['트리클로로에틸렌'],
 '틱소트로픽 인덱스': ['틱소트로픽 인덱스'],
 '파단 후 신장량': ['파단 후 신장량'],
 '파단시 신장률': ['파단시 신장률'],
 '파단하중': ['파단하중'],
 '파쇄면 2면 이상 비율': ['파쇄면 2면 이상 비율'],
 '파쇄면 비율': ['파쇄면 비율'],
 '파쇄시험': ['파쇄시험'],
 '파열강도': ['파열강도'],
 '팽윤 후 두께': ['팽윤 후 두께'],
 '팽윤도': ['팽윤도'],
 '팽창 높이': ['팽창 높이'],
 '팽창률': ['팽창률'],
 '팽창성': ['팽창성'],
 '팽창율': ['팽창율'],
 '페놀': ['페놀'],
 '페놀류': ['페놀류'],
 '편경차': ['편경차'],
 '편장석률': ['편장석률'],
 '편평 세장편 함유량': ['편평 세장편 함유량'],
 '편평성': ['편평성'],
 '편평시험': ['편평시험'],
 '평균 바깥지름': ['평균 바깥지름'],
 '평균 안지름': ['평균 안지름'],
 '평균길이': ['평균길이'],
 '평균입자크기': ['평균입자크기'],
 '평누름에 의한 압축 하중': ['평누름에 의한 압축 하중'],
 '평량': ['평량'],
 '평면 인장 강도': ['평면 인장 강도'],
 '평탄성': ['평탄성'],
 '평판재하': ['평판재하'],
 '평판재하시험': ['평판재하시험'],
 '포화도': ['포화도'],
 '폭': ['폭'],
 '폴리클로리네이티드비페닐': ['폴리클로리네이티드비페닐'],
 '폼알데하이드 방출량': ['폼알데하이드 방출량', '폼알데하이드 방산량'],
 '표건밀도': ['표건밀도'],
 '표면 투명층 두께': ['표면 투명층 두께'],
 '표면결함검사': ['표면결함검사'],
 '표면경도': ['표면경도'],
 '표면층 두께': ['표면층 두께'],
 '표준지름': ['표준지름'],
 '표층용 - 공극률': ['표층용 - 공극률'],
 '표층용 - 다짐회수': ['표층용 - 다짐회수'],
 '표층용 - 변형강도': ['표층용 - 변형강도'],
 '표층용 - 안정도': ['표층용 - 안정도', '표층용 - 동적안정도'],
 '표층용 - 인장강도비': ['표층용 - 인장강도비'],
 '표층용 - 포화도': ['표층용 - 포화도'],
 '표층용 - 흐름값': ['표층용 - 흐름값'],
 '프루프롤링': ['프루프롤링'],
 '프탈레이트계': ['프탈레이트계'],
 '플로': ['플로'],
 '플로값 비': ['플로값 비'],
 '플로우값': ['플로우값', '플로값'],
 '피로반복시험': ['피로반복시험'],
 '피막박리': ['피막박리'],
 '피막의 성질': ['피막의 성질'],
 '피복두께': ['피복두께'],
 '피복두께 확인': ['피복두께 확인'],
 '핀홀': ['핀홀'],
 '핀홀시험': ['핀홀시험'],
 '필터재-인열강도': ['필터재-인열강도', '필터재-인장강도'],
 '하중변형온도': ['하중변형온도'],
 '하중재하, 균열검사': ['하중재하, 균열검사'],
 '함수비': ['함수비'],
 '함수율': ['함수율'],
 '함수율, 흡수성': ['함수율, 흡수성'],
 '합성입도': ['합성입도'],
 '항복 강도': ['항복 강도', '항복강도'],
 '항복강도 판-두께 20 mm 이하': ['항복강도 판-두께 20 mm 이하'],
 '항복비': ['항복비'],
 '항복점': ['항복점'],
 '항복점 또는 내력, 인장강도, 연신율': ['항복점 또는 내력, 인장강도, 연신율'],
 '항복점 또는 항복 강도': ['항복점 또는 항복 강도', '항복점 또는 0.2%항복 강도'],
 '항복점 또는 항복 강도, 인장강도, 연신율': ['항복점 또는 항복 강도, 인장강도, 연신율'],
 '항장적': ['항장적'],
 '해당시험 미진행': ['해당시험 미진행'],
 '허용변위시험 크기 1,960 × 1,000 mm 이하': ['허용변위시험 크기 1,960 × 1,000 mm 이하', '허용변위시험 크기 3,960 × 1,000 mm 이하'],
 '허용압축응력': ['허용압축응력'],
 '현미경조직시험': ['현미경조직시험'],
 '현장밀도': ['현장밀도'],
 '현장밀도 또는 포화도': ['현장밀도 또는 포화도'],
 '현장배합수정': ['현장배합수정'],
 '현장투수능력': ['현장투수능력'],
 '현탁 물질의 양': ['현탁 물질의 양'],
 '형상': ['형상'],
 '혼용률': ['혼용률'],
 '혼합 종결 확인 용이성': ['혼합 종결 확인 용이성'],
 '혼합물 온도': ['혼합물 온도'],
 '혼합성': ['혼합성'],
 '화학분석': ['화학분석'],
 '화학성분': ['화학성분'],
 '확대기초': ['확대기초'],
 '확산 반사율': ['확산 반사율'],
 '확인시험': ['확인시험'],
 '활성도 지수': ['활성도 지수'],
 '황산구리': ['황산구리'],
 '회복률': ['회복률'],
 '회복율': ['회복율'],
 '회분': ['회분'],
 '회분 시험': ['회분 시험'],
 '회분질량': ['회분질량'],
 '회전점도': ['회전점도'],
 '횡방향': ['횡방향'],
 '횡방향 측면 굽힘시험': ['횡방향 측면 굽힘시험'],
 '후의 침입도비': ['후의 침입도비'],
 '휘발성유기화합물함유량': ['휘발성유기화합물함유량'],
 '휠 트래킹': ['휠 트래킹'],
 '휠 트랙킹': ['휠 트랙킹'],
 '휨 강도': ['휨 강도'],
 '휨 강도, 습윤시 휨 강도': ['휨 강도, 습윤시 휨 강도'],
 '휨 강도, 흡수율': ['휨 강도, 흡수율'],
 '휨 강성 변형량': ['휨 강성 변형량'],
 '휨 시험': ['휨 시험'],
 '휨 파괴 하중': ['휨 파괴 하중', '휨파괴하중'],
 '휨 하중': ['휨 하중', '휨하중', '내 휨하중'],
 '휨, 압축, 부착강도': ['휨, 압축, 부착강도'],
 '흐름값': ['흐름값'],
 '흐름성': ['흐름성'],
 '흐름손신률': ['흐름손신률'],
 '흐름손실률': ['흐름손실률'],
 '흐름시험': ['흐름시험'],
 '흐름저항성': ['흐름저항성'],
 '흘러내림 저항': ['흘러내림 저항'],
 '흘러내림 저항성능-주름발생': ['흘러내림 저항성능-주름발생'],
 '흘러내림 저항성능-흘러내림길이': ['흘러내림 저항성능-흘러내림길이'],
 '흙 입자의 밀도': ['흙 입자의 밀도'],
 '흙의 공학적 분류': ['흙의 공학적 분류'],
 '흙의 압밀시험': ['흙의 압밀시험'],
 '흙의 투수시험': ['흙의 투수시험'],
 '흡수 두께 팽창률': ['흡수 두께 팽창률'],
 '흡수량': ['흡수량'],
 '흡수성': ['흡수성'],
 '흡수시 내박리성': ['흡수시 내박리성', '흡수시내박리성'],
 '흡수에 의한 길이 변화율': ['흡수에 의한 길이 변화율', '흡수에 의한 길이변화율'],
 '흡수율': ['흡수율'],
 '흡음률': ['흡음률'],
 '희석 안정성, 내수성': ['희석 안정성, 내수성']}